In order to run this notebook, the following environment and packages had to be installed:

$ conda create -n v-env python=3.8

activate v-env

conda activate v-env

conda config --set channel_priority false

conda install -c conda-forge prophet

pip install salesforce-merlion

pip install pingouin

pip install PyYAML

pip install SQLAlchemy

pip install ipykernel

pip install jinja2

pip install sfc-merlion[plot]

pip install -e ts_datasets/

conda install -e ts_datasets/

conda install -c conda-forge lightgbm

conda activate v-env

pip install -e ts_datasets/

pip intall -e ./Merlion/ts_dataset/

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import norm

#from ts_datasets.anomaly import NAB
#from ts_datasets.forecast import M4

#from merlion.plot import plot_anoms
from merlion.utils import TimeSeries
from merlion.utils.time_series import TimeSeries

from merlion.transform.base import Identity
from merlion.transform.resample import TemporalResample
from merlion.transform.moving_average import DifferenceTransform

from merlion.models.defaults import DefaultDetectorConfig, DefaultDetector
#from merlion.models.anomaly.isolation_forest import IsolationForest, IsolationForestConfig
#from merlion.models.anomaly.windstats import WindStats, WindStatsConfig
#from merlion.models.anomaly.forecast_based.prophet import ProphetDetector, ProphetDetectorConfig

from merlion.models.forecast.arima import Arima, ArimaConfig
from merlion.models.forecast.prophet import Prophet, ProphetConfig
from merlion.models.forecast.smoother import MSES, MSESConfig

#from merlion.models.ensemble.anomaly import DetectorEnsemble, DetectorEnsembleConfig
from merlion.models.defaults import DefaultForecasterConfig, DefaultForecaster
from merlion.models.ensemble.combine import Mean, ModelSelector
from merlion.models.ensemble.forecast import ForecasterEnsemble, ForecasterEnsembleConfig

#from merlion.evaluate.anomaly import TSADMetric
from merlion.post_process.threshold import AggregateAlarms
from merlion.evaluate.forecast import ForecastMetric

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-g_vmdl2b because the default path (/home/mpingol/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [11]:
from merlion.evaluate.forecast import ForecastMetric
from merlion.models.factory import ModelFactory
from merlion.models.ensemble.combine import ModelSelector

# 1 Day Close and Tone 

In [6]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/facebook_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

In [7]:
tesla_mv = tesla[['Close', 'Daily Average Tone']]

In [8]:
tesla_mv

,Close,Daily Average Tone
Date,,
2015-02-18,76.709999,-2.857143
2015-02-19,79.419998,-3.970588
2015-02-20,79.900002,-4.884259
2015-02-21,78.839996,-3.934307
2015-02-22,78.449997,-2.397436
...,...,...
2017-12-27,177.820007,-3.890591
2017-12-28,177.509995,-3.319239
2017-12-29,179.100006,-3.135514


In [9]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

838 points in train split, 210 points in test split.
Time series is 2-dimensional


In [12]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:57<00:00, 313536.58it/s]


RMSE:  7.3408
sMAPE: 3.2859
MAE:   5.4718

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:07<00:00, 2537938.58it/s]


RMSE:  5.9331
sMAPE: 2.2593
MAE:   3.6666



# 1 Day Close Only

In [13]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/facebook_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

In [14]:
tesla_mv = tesla[['Close']]

In [15]:
tesla_mv

,Close
Date,
2015-02-18,76.709999
2015-02-19,79.419998
2015-02-20,79.900002
2015-02-21,78.839996
2015-02-22,78.449997
...,...
2017-12-27,177.820007
2017-12-28,177.509995
2017-12-29,179.100006


In [16]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

838 points in train split, 210 points in test split.
Time series is 1-dimensional


In [17]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:09<00:00, 1883372.40it/s]


RMSE:  7.8516
sMAPE: 3.1905
MAE:   5.1970

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:06<00:00, 2995525.95it/s]


RMSE:  5.9331
sMAPE: 2.2593
MAE:   3.6666



# 3 day close and tone

In [18]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/facebook_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close', 'Daily Average Tone']]

tesla_mv

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 70
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="3d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

838 points in train split, 210 points in test split.
Time series is 2-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [01:04<00:00, 278519.06it/s]


RMSE:  5.7315
sMAPE: 3.2136
MAE:   5.3544

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:07<00:00, 2370070.45it/s]


RMSE:  4.0409
sMAPE: 2.2332
MAE:   3.6291



# 3 day close only

In [19]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/facebook_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close']]

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

838 points in train split, 210 points in test split.
Time series is 1-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:09<00:00, 1923368.44it/s]


RMSE:  7.8516
sMAPE: 3.1905
MAE:   5.1970

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:06<00:00, 2988235.38it/s]


RMSE:  5.9331
sMAPE: 2.2593
MAE:   3.6666



# 5 day close and tone

In [22]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/facebook_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close', 'Daily Average Tone']]

tesla_mv

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 42
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="5d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

838 points in train split, 210 points in test split.
Time series is 2-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [01:19<00:00, 228295.99it/s]


RMSE:  6.7331
sMAPE: 3.7117
MAE:   6.1568

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:07<00:00, 2436339.91it/s]


RMSE:  5.1371
sMAPE: 2.7643
MAE:   4.5122



# 5 day close only

In [23]:
tesla = pd.read_csv('~/cpt1/bdcc/final_project/GDELTS/facebook_completed.csv')
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close']]

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 42
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="5d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

838 points in train split, 210 points in test split.
Time series is 1-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:10<00:00, 1788011.37it/s]


RMSE:  7.1031
sMAPE: 3.9162
MAE:   6.3890

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:06<00:00, 2872280.82it/s]


RMSE:  5.1371
sMAPE: 2.7643
MAE:   4.5122

